In [11]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import ultralytics

In [18]:

from roboflow import Roboflow
rf = Roboflow(api_key="iqWudABIjfXqATDPAmLo")
project = rf.workspace("custome-yolo").project("all-weapons-data-jqimy")
version = project.version(1)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to All-weapons-data-1 in yolov8::  15%|█▌        | 2585/17172 [00:05<00:30, 477.29it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'All-weapons-data-1\\train\\images\\Afghan_Uniformed_Police_officers_fire_their_AK-47_assault_rifles_under_the_watch_of_Czech_Military_Police_instructors-_left-_on_Swanson_Small_Arms_Range_during_training_at_Forward_Operating_Base_Shank-_Logar_120201-.rf.3c7b012d85b7570eb22debc7709b0859.jpg'

In [12]:
train_images_path="/content/drive/MyDrive/All-weapons-data-1/train/images"
train_labels_path="/content/drive/MyDrive/All-weapons-data-1/train/labels"
test_images_path="/content/drive/MyDrive/All-weapons-data-1/test/images"
test_labels_path="/content/drive/MyDrive/All-weapons-data-1/test/labels"
validation_images_path="/content/drive/MyDrive/All-weapons-data-1/valid/images"
validation_labels_path="/content/drive/MyDrive/All-weapons-data-1/valid/labels"

We will first check if all the train , test and valid set have correct corresponding images and labels or not


In [13]:

def check_image_label_pairs(images_path, labels_path):
    image_names = {os.path.splitext(f)[0] for f in os.listdir(images_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))}
    label_names = {os.path.splitext(f)[0] for f in os.listdir(labels_path) if f.lower().endswith('.txt')}

    missing_labels = image_names - label_names
    missing_images = label_names - image_names

    print(f"Total images: {len(image_names)}")
    print(f"Total labels: {len(label_names)}")
    print("")

    if missing_labels:
        print(f"{len(missing_labels)} images have no label file:")
        for name in sorted(list(missing_labels))[:10]:
            print("  -", name)
        if len(missing_labels) > 10:
            print(f"  ... and {len(missing_labels)-10} more.")
    else:
        print("All images have matching labels.")

    print("")

    if missing_images:
        print(f"{len(missing_images)} labels have no image file:")
        for name in sorted(list(missing_images))[:10]:
            print("  -", name)
        if len(missing_images) > 10:
            print(f"  ... and {len(missing_images)-10} more.")
    else:
        print("All labels have matching images.")


In [14]:
check_image_label_pairs(train_images_path, train_labels_path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/All-weapons-data-1/train/images'

In [ ]:
check_image_label_pairs(test_images_path, test_labels_path)

Total images: 593
Total labels: 593

All images have matching labels.

All labels have matching images.


In [ ]:
check_image_label_pairs(validation_images_path, validation_labels_path)

Total images: 591
Total labels: 591

All images have matching labels.

All labels have matching images.


We dont have to worry about dimensions , yolo will take care of it automatically.

In [ ]:
import os
from collections import Counter

label_dir = "../train/labels"  # adjust path if needed
all_labels = []

for file in os.listdir(label_dir):
    if file.endswith(".txt"):
        with open(os.path.join(label_dir, file)) as f:
            for line in f:
                class_id = int(line.split()[0])
                all_labels.append(class_id)

print(Counter(all_labels))


In [ ]:
from ultralytics import YOLO

In [ ]:

model = YOLO("yolov8n.yaml")  # just the architecture, no weights
model.train(data="/content/All-weapons-data-1/data.yaml",name="weapons_train",project="/content/drive/MyDrive/Colab Notebooks/YOLO", epochs=400, imgsz=640)
